## GCM Compute Fluxes
This notebook loads in simulations from MiMA at $2\degree$ resolution. Here we assume that the data is available at a 4x daily temporal resolution. 

It then runs the AD99 offline parameterization over the data to predict both the zonal and meridional momentum fluxes before saving it to the drive

In [25]:
### Master Variables
dc = 0.4 # Spectral resolution
exclude_unbroken = False # Exclude waves that are unbroken at top level from the momentum flux calculations? 
use_intrinsic_c = 'always' # always center distributon on zero intrinsic phase speed vs never (center on zero ground relative speed) or switch at latitude ? 
Fs0 = 4.3e-3 # Pa, initial momentum flux at source level
cw = 35 # m/s, gravity wave phase speed source distribution half width at half maximum 
launch_level_height = 1000 # in m 
expname=f'{use_intrinsic_c}_{"exclude" if exclude_unbroken else "include"}_unbroken_1year'
if launch_level_height != 9000: 
    expname=f'll{int(launch_level_height)}_{expname}'
expname

'll1000_always_include_unbroken_1year'

Setup dask distributed if needed

In [2]:
from dask_jobqueue import SLURMCluster
NCORES = 8
NPROCESS = 8
NCORESPERPROCESS = NCORES//NPROCESS
constraints = ['-C \"CLASS:SH4_CBASE|CLASS:SH4_CPERF\"'] # SH4 nodes are the fastest, and mixing node gens seems to cause ib0 issues.
cluster = SLURMCluster(queue='serc',memory='48GiB',cores=NCORES,processes=NPROCESS,walltime='06:00:00',job_extra_directives=constraints,log_directory='/scratch/users/robcking/dask_worker_logs')
cluster.scale(jobs=10) # roughly but tune to scheduler 
cluster



Dashboard: http://10.20.3.15:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.20.3.15:39542,Workers: 0
Dashboard: http://10.20.3.15:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [3]:
from dask.distributed import Client

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.20.3.15:8787/status,
Dashboard: http://10.20.3.15:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.20.3.15:39542,Workers: 0
Dashboard: http://10.20.3.15:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


Load data

In [26]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from ad99py.ad99 import AlexanderDunkerton1999
from ad99py.masks import mask_dataset,load_mask
from ad99py.constants import GRAV,R_DRY,C_P


DATA_PATH = 'data' # Load in raw data from GCM, here we use MiMA that has been interpolated to the ERA5 pressure levels. 
ds = xr.open_dataset(os.path.join(DATA_PATH,'atmos_4xdaily_interp_1year.nc'),chunks={'time':8})
ds = ds[['ucomp','temp','vcomp','height']]
ds = ds.isel(level=slice(None,None,-1))
ds = ds.rename(lat='latitude',lon='longitude')
ds = ds.transpose('time','longitude','latitude','level')
ds = ds.chunk({'time':8})
ds

<xarray.Dataset> Size: 7GB
Dimensions:    (time: 1440, level: 37, latitude: 64, longitude: 128)
Coordinates:
  * longitude  (longitude) float32 512B 0.0 2.812 5.625 ... 351.6 354.4 357.2
  * latitude   (latitude) float32 256B -87.86 -85.1 -82.31 ... 82.31 85.1 87.86
  * level      (level) float32 148B 1e+03 975.0 950.0 925.0 ... 5.0 3.0 2.0 1.0
  * time       (time) object 12kB 0025-01-01 00:00:00 ... 0025-12-30 18:00:00
Data variables:
    ucomp      (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    temp       (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    vcomp      (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    height     (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
Attributes:
    filename:     atmos_4xdaily_interp.nc
    title:        MiMA
    history:      Tue Jul 08 18:56:01 2025: plevel.sh -a -i atmos_4xdaily_fla...
    NCO:          netCDF Operators version 5.2.4 (Homepage = http://nco.sf.ne...
    fre-nctools:  FRE NCTools version 2024.05 (git: 2024.05.01-3-gffa6ce6)

Extract core fields

In [27]:
us = ds.ucomp.data
vs = ds.vcomp.data
zs = ds.height.data
temps = ds.temp.data
level = ds.level.data

Calculate required variables for AD99

In [28]:
import dask.array as da
from ad99py.variables import bouyancy_freq_squared,density

Ns = bouyancy_freq_squared(temps,zs)**0.5
rho = density(temps,level)
lat=da.broadcast_to(ds.latitude.data[None,None,:].data,us.shape[:-1])[...,None].rechunk((8,-1,-1,-1))
lat

dask.array<rechunk-merge, shape=(1440, 128, 64, 1), dtype=float32, chunksize=(8, 128, 64, 1), chunktype=numpy.ndarray>

In [29]:
from ad99py import AlexanderDunkerton1999

ad99 = AlexanderDunkerton1999(Fs0=4.3e-3,
                            cw=35,
                            exclude_unbroken=exclude_unbroken,
                            use_intrinsic_c=use_intrinsic_c,
                            dc=dc,
                            source_level_height=launch_level_height
) # this is following the MiMA spec 
ad99

/home/groups/aditis2/robcking/ad99py/src/ad99py/ad99.py:64: UserWarning: `source` is not set, using default Gaussian source spectrum, with `cw=35` and `Bm=0.4`.
  warn(f"`source` is not set, using default Gaussian source spectrum, with `cw={cw}` and `Bm={Bm}`.")


AlexanderDunkerton1999(Fs0=0.0043,height=1000, λ=300000.0, damping_level=None)

In [30]:
def dask_ad99_map_block(ublock,Nblock,zblock,rhoblock,latblock,ad99=None):
    """
    Map indivudal blocks by linearly running them through the parameterization. 
    Not very fast or efficient right now but could be optimized further in future
    """
    batch_shape = ublock.shape[:-1]
    nlevels = ublock.shape[-1]
    
    ublock_flt = ublock.reshape((-1,nlevels))
    Nblock_flt = Nblock.reshape((-1,nlevels))
    zblock_flt = zblock.reshape((-1,nlevels))
    rhoblock_flt = rhoblock.reshape((-1,nlevels))
    latblock_flt = latblock.ravel()
    results = np.array([
        ad99.momentum_flux_neg_ptv(u,N,z,rho,lat) for u,N,z,rho,lat in zip(ublock_flt,Nblock_flt,zblock_flt,rhoblock_flt,latblock_flt)
    ])
    
    result_shp = results.reshape((*batch_shape,2,nlevels))
    result_rtn = np.moveaxis(result_shp,-2,-1)
    return result_rtn 

In [31]:
from functools import partial
result_u = da.map_blocks(
    partial(dask_ad99_map_block,ad99=ad99),
    us,Ns,zs,
    rho,
    lat,
    new_axis=len(us.shape),
    dtype=us.dtype,
    chunks=tuple(c[0] for c in us.chunks) + (2,))

result_v = da.map_blocks(
    partial(dask_ad99_map_block,ad99=ad99),
    vs,Ns,zs,
    rho,
    lat,
    new_axis=len(vs.shape),
    dtype=vs.dtype,
    chunks=tuple(c[0] for c in vs.chunks) + (2,))

ntv_u_flux = result_u[...,0]
ptv_u_flux = result_u[...,1]
ntv_v_flux = result_v[...,0]
ptv_v_flux = result_v[...,1]

In [32]:
ds_new = ds.copy()
ds_new = ds_new.rename(ucomp='u',vcomp='v',height='z')
dims = ('time','longitude','latitude','level')
ds_new['Ns'] = (dims, Ns)
ds_new['rho'] = (dims, rho)
ds_new['gw_flux_westward'] = (dims, ntv_u_flux)
ds_new['gw_flux_eastward'] = (dims, ptv_u_flux)
ds_new['gw_flux_southward'] = (dims, ntv_v_flux)
ds_new['gw_flux_northward'] = (dims, ptv_v_flux)
ds_new


<xarray.Dataset> Size: 17GB
Dimensions:            (time: 1440, longitude: 128, latitude: 64, level: 37)
Coordinates:
  * longitude          (longitude) float32 512B 0.0 2.812 5.625 ... 354.4 357.2
  * latitude           (latitude) float32 256B -87.86 -85.1 ... 85.1 87.86
  * level              (level) float32 148B 1e+03 975.0 950.0 ... 3.0 2.0 1.0
  * time               (time) object 12kB 0025-01-01 00:00:00 ... 0025-12-30 ...
Data variables:
    u                  (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    temp               (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    v                  (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    z                  (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    Ns                 (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    rho                (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    gw_flux_westward   (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    gw_flux_eastward   (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    gw_flux_southward  (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    gw_flux_northward  (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
Attributes:
    filename:     atmos_4xdaily_interp.nc
    title:        MiMA
    history:      Tue Jul 08 18:56:01 2025: plevel.sh -a -i atmos_4xdaily_fla...
    NCO:          netCDF Operators version 5.2.4 (Homepage = http://nco.sf.ne...
    fre-nctools:  FRE NCTools version 2024.05 (git: 2024.05.01-3-gffa6ce6)

In [33]:
ds_new.attrs['ad99_intrinsic_c_settings'] = 'always'
ds_new.attrs['ad99_Bt'] = 4.3e-3
ds_new.attrs['ad99_cw'] = 35
ds_new.attrs['ad99_Bm'] = dc
ds_new.attrs['ad99_exclude_unbroken'] = int(exclude_unbroken)
ds_new.attrs['ad99_launch_level'] =launch_level_height
ds_new.attrs['ad99_source'] = 'gaussian'
ds_new

<xarray.Dataset> Size: 17GB
Dimensions:            (time: 1440, longitude: 128, latitude: 64, level: 37)
Coordinates:
  * longitude          (longitude) float32 512B 0.0 2.812 5.625 ... 354.4 357.2
  * latitude           (latitude) float32 256B -87.86 -85.1 ... 85.1 87.86
  * level              (level) float32 148B 1e+03 975.0 950.0 ... 3.0 2.0 1.0
  * time               (time) object 12kB 0025-01-01 00:00:00 ... 0025-12-30 ...
Data variables:
    u                  (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    temp               (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    v                  (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    z                  (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    Ns                 (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    rho                (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    gw_flux_westward   (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    gw_flux_eastward   (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    gw_flux_southward  (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
    gw_flux_northward  (time, longitude, latitude, level) float32 2GB dask.array<chunksize=(8, 128, 64, 37), meta=np.ndarray>
Attributes:
    filename:                   atmos_4xdaily_interp.nc
    title:                      MiMA
    history:                    Tue Jul 08 18:56:01 2025: plevel.sh -a -i atm...
    NCO:                        netCDF Operators version 5.2.4 (Homepage = ht...
    fre-nctools:                FRE NCTools version 2024.05 (git: 2024.05.01-...
    ad99_intrinsic_c_settings:  always
    ad99_Bt:                    0.0043
    ad99_cw:                    35
    ad99_Bm:                    0.4
    ad99_exclude_unbroken:      0
    ad99_launch_level:          1000
    ad99_source:                gaussian

In [34]:
OUTPUT_PATH = f'{DATA_PATH}/mima_gwf_{expname}.nc'
OUTPUT_PATH
ds_new.to_netcdf(OUTPUT_PATH)